<a href="https://colab.research.google.com/github/AN-Akhand/Transaction-Data-Clustering-with-NLP/blob/main/nlp_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [2]:
from pprint import pprint

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv("/content/drive/MyDrive/Transaction_Naration_Data_Set.csv")

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
data['NARATION'].iloc[[21886]]

21886    CAP (Smart App), NPSB Fund Transfer. Trf to ,SOCIAL ISLAMI BANK LTD fr BA
Name: NARATION, dtype: object

In [ ]:
data.head(15)

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
valid_words = set(nltk.corpus.words.words())
stopwords = stopwords.words('english')
stopwords.extend(['no', 'c', 'ae', 'st', 'palli', 'b', 'beauty', 'bazaar', 'id',
                  'hat', 'x', 'l', 'bu', 'roll', 'amount', 'bill', 'branch',
                  'salary', 'amt', 'account', 'bank', 'office', 'al', 'mamum', 
                  'hasan', 'hossain', 'mar', 'amt', 'islami', 'sonali', 'nid', 
                  'itna', 'govt', 'usd', 'miah', 'uddin', 'eid', 'ul', 'fitr', 
                  'ac', 'title', 'begum', 'para', 'title', 'shahin', 'per', 
                  'month', 'k', 'khan'])
x = []
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w, 'v') for w in w_tokenizer.tokenize(text)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [5]:
nar = pd.DataFrame(data.loc[:, 'NARATION'])
nar = nar.drop_duplicates()

In [6]:
nar.loc[:,'NARATION']=nar['NARATION'].str.lower()
nar.loc[:,'NARATION'] = nar.loc[:,'NARATION'].str.replace('withdrawal', 'withdraw')
nar.loc[:,'NARATION'] = nar['NARATION'].str.replace('[^A-Za-z]+', ' ', regex=True)
nar = nar.replace(['^\s+$'], np.nan, regex = True)
nar = nar[nar['NARATION'].notnull()]
nar.loc[:,'NARATION'] = nar.loc[:,'NARATION'].apply(lambda words: ' '.join(word.lower() for word in words.split(' ') if word not in stopwords))
nar.loc[:,'NARATION'] = nar.loc[:,'NARATION'].apply(lambda words: ' '.join(word.lower() for word in words.split(' ') if word in valid_words))
nar.loc[:,'lem_tok'] = nar.loc[:,'NARATION'].apply(lemmatize_text)
nar = nar[nar['lem_tok'].map(lambda d: len(d)) > 0]

In [7]:
nar = nar.drop_duplicates(subset=['NARATION'])

In [ ]:
nar

In [14]:
test = nar.sample(1000, random_state=1234)

In [22]:
for i in range(10):  
  id2word = corpora.Dictionary(test['lem_tok'])
  texts = test['lem_tok']
  corpus = [id2word.doc2bow(text) for text in texts]
  [[(id2word[id], freq) for id, freq in cp] for cp in corpus]
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=i + 1, 
                                             random_state=100,
                                             update_every=1,
                                             chunksize=100,
                                             passes=8,
                                             alpha='auto',
                                             per_word_topics=True)
  doc_lda = lda_model[corpus]
  coherence_model_lda = CoherenceModel(model=lda_model, texts=test['lem_tok'], dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nNo of clusters: ', i + 1)
  print('\nCoherence Score: ', coherence_lda)


No of clusters:  1

Coherence Score:  0.3840496668825152

No of clusters:  2

Coherence Score:  0.49313866081938

No of clusters:  3

Coherence Score:  0.5337162922194788

No of clusters:  4

Coherence Score:  0.6025630924708253

No of clusters:  5

Coherence Score:  0.6314968909280414

No of clusters:  6

Coherence Score:  0.5853152138079539

No of clusters:  7

Coherence Score:  0.615862637540845

No of clusters:  8

Coherence Score:  0.6177622767947615

No of clusters:  9

Coherence Score:  0.6220134339484906

No of clusters:  10

Coherence Score:  0.6080859768088855


In [23]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=8,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())

[(0,
  '0.060*"loan" + 0.054*"fee" + 0.047*"disbursement" + 0.038*"transaction" + '
  '0.022*"allowance" + 0.020*"charge" + 0.019*"wrongly" + 0.017*"outlet" + '
  '0.014*"monthly" + 0.013*"rectification"'),
 (1,
  '0.143*"transfer" + 0.139*"fund" + 0.094*"smart" + 0.059*"regular" + '
  '0.050*"cap" + 0.030*"payment" + 0.020*"doc" + 0.016*"march" + 0.015*"rent" '
  '+ 0.013*"transaction"'),
 (2,
  '0.039*"sod" + 0.027*"das" + 0.027*"post" + 0.027*"abu" + 0.021*"need" + '
  '0.019*"amin" + 0.018*"time" + 0.016*"cost" + 0.015*"support" + 0.013*"tax"'),
 (3,
  '0.110*"credit" + 0.050*"cheque" + 0.047*"bank" + 0.039*"agent" + '
  '0.018*"limit" + 0.017*"ad" + 0.015*"road" + 0.015*"client" + 0.014*"gram" + '
  '0.013*"purchase"'),
 (4,
  '0.174*"cash" + 0.123*"deposit" + 0.115*"withdraw" + 0.060*"agent" + '
  '0.050*"inter" + 0.025*"purpose" + 0.014*"n" + 0.013*"due" + 0.013*"name" + '
  '0.010*"return"')]


In [ ]:
nar.to_csv("/content/drive/MyDrive/Pre_Processed_Transaction_Naration_Data_Set.csv")

In [ ]:
sample = data.sample(5000, random_state=123)
sample.loc[:,'NARATION']=sample['NARATION'].str.lower()
sample.loc[:,'NARATION'] = sample.loc[:,'NARATION'].str.replace('withdrawal', 'withdraw')
sample.loc[:,'NARATION'] = sample.loc[:,'NARATION'].str.replace('[^A-Za-z0-9]+', ' ', regex=True)
sample.loc[:,'NARATION'] = sample.loc[:,'NARATION'].apply(lambda words: ' '.join(word.lower() for word in words.split(' ') if word not in stopwords))
sample.loc[:,'lemmatized_tokens'] = sample.loc[:,'NARATION'].apply(lemmatize_text)

In [ ]:
sample.head(50)